# Notebook 03: Routing Simulation Development

This notebook develops and tests routing simulation strategies for NEMT operations.

## Table of Contents
1. [Setup & Data Loading](#setup)
2. [Strategy Implementations](#strategies)
3. [Simulation Execution](#simulation)
4. [Strategy Comparison](#comparison)
5. [Sensitivity Analysis](#sensitivity)
6. [Key Findings](#findings)

<a id="setup"></a>
## 1. Setup & Data Loading

In [13]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path
from importlib import reload

# Add parent to path for local imports
sys.path.insert(0, str(Path.cwd().parent))

from src.config import RANDOM_SEED, PROCESSED_DIR
from src.utils import set_seed, haversine_distance
import src.routing_simulation
reload(src.routing_simulation)  # Force reload to pick up changes
from src.routing_simulation import (
    assign_fcfs,
    assign_nearest,
    assign_capacity_aware,
    simulate_strategy,
    run_simulation_comparison,
    SimulationResult
)

# Set reproducible seed
set_seed(RANDOM_SEED)

# Plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 20)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✅ Setup complete")

✅ Setup complete


In [14]:
# Load processed trip data from previous notebook
trips_df = pd.read_csv(
    PROCESSED_DIR / 'trips_with_efficiency.csv', 
    parse_dates=['requested_pickup_time', 'scheduled_pickup_time', 
                 'actual_pickup_time', 'actual_dropoff_time']
)
drivers_df = pd.read_csv(PROCESSED_DIR / 'drivers.csv')

print(f"✅ Loaded {len(trips_df):,} trips")
print(f"✅ Loaded {len(drivers_df):,} drivers")
print(f"📊 Regions: {trips_df['region'].nunique()}")
print(f"📅 Date range: {trips_df['scheduled_pickup_time'].min().date()} to {trips_df['scheduled_pickup_time'].max().date()}")

✅ Loaded 5,000 trips
✅ Loaded 150 drivers
📊 Regions: 5
📅 Date range: 2025-01-01 to 2025-03-30


<a id="strategies"></a>
## 2. Strategy Implementations

### Overview of Routing Strategies

| Strategy | Description | Complexity |
|----------|-------------|------------|
| **FCFS** | First-Come-First-Served - Assign in order received | O(n) |
| **Nearest** | Assign to closest available driver | O(n²) |
| **Capacity-Aware** | Balance load across drivers considering utilization | O(n²) |

> **Note**: These strategies simulate historical comparisons, not real-time VRP optimization.

In [5]:
# Prepare sample data for strategy demonstration
sample_trips = trips_df[~trips_df['is_cancelled']].head(100).copy()
sample_drivers = drivers_df.head(20).copy()

print(f"Demo dataset: {len(sample_trips)} trips, {len(sample_drivers)} drivers")

Demo dataset: 100 trips, 20 drivers


In [7]:
# Strategy 1: FCFS Assignment
driver_ids = sample_drivers['driver_id'].tolist()
fcfs_result = assign_fcfs(sample_trips, driver_ids)
print("FCFS Assignment Results:")
print(f"  Assignments made: {len(fcfs_result)}")
print(f"  Unique drivers used: {fcfs_result['assigned_driver'].nunique()}")
print(f"\nDriver assignment distribution:")
print(fcfs_result['assigned_driver'].value_counts().head(10))

FCFS Assignment Results:
  Assignments made: 100
  Unique drivers used: 3

Driver assignment distribution:
assigned_driver
DRV_0000    95
DRV_0001     4
DRV_0002     1
Name: count, dtype: int64


In [8]:
# Strategy 2: Nearest Distance Assignment
# Create driver locations (simulated at random positions initially)
driver_locations = {
    d: (
        sample_trips['pickup_lat'].mean() + np.random.uniform(-0.1, 0.1),
        sample_trips['pickup_lng'].mean() + np.random.uniform(-0.1, 0.1)
    )
    for d in driver_ids
}
nearest_result = assign_nearest(sample_trips, driver_ids, driver_locations)
print("Nearest Distance Assignment Results:")
print(f"  Assignments made: {len(nearest_result)}")
print(f"  Unique drivers used: {nearest_result['assigned_driver'].nunique()}")
print(f"\nDriver assignment distribution:")
print(nearest_result['assigned_driver'].value_counts().head(10))

Nearest Distance Assignment Results:
  Assignments made: 100
  Unique drivers used: 17

Driver assignment distribution:
assigned_driver
DRV_0017    16
DRV_0014    10
DRV_0003     9
DRV_0001     8
DRV_0002     7
DRV_0012     6
DRV_0006     5
DRV_0005     5
DRV_0019     5
DRV_0016     5
Name: count, dtype: int64


In [9]:
# Strategy 3: Capacity-Aware Assignment
# Create driver capacities based on typical vehicle sizes
driver_capacities = {d: np.random.choice([4, 6, 8]) for d in driver_ids}
capacity_result = assign_capacity_aware(sample_trips, driver_ids, driver_capacities)
print("Capacity-Aware Assignment Results:")
print(f"  Assignments made: {len(capacity_result)}")
print(f"  Unique drivers used: {capacity_result['assigned_driver'].nunique()}")
print(f"\nDriver assignment distribution:")
print(capacity_result['assigned_driver'].value_counts().head(10))

Capacity-Aware Assignment Results:
  Assignments made: 100
  Unique drivers used: 2

Driver assignment distribution:
assigned_driver
DRV_0001    95
DRV_0005     5
Name: count, dtype: int64


<a id="simulation"></a>
## 3. Simulation Execution

Running full simulation comparison across all strategies.

In [15]:
# Run full simulation comparison
# Uses non-cancelled trips only
active_trips = trips_df[~trips_df['is_cancelled']].copy()
num_drivers = len(drivers_df)

print(f"Running simulation on {len(active_trips):,} trips with {num_drivers} drivers...")
simulation_results = run_simulation_comparison(active_trips, num_drivers=num_drivers)

print("\n✅ Simulation Complete!")
print(f"Strategies evaluated: {len(simulation_results)}")

Running simulation on 4,853 trips with 150 drivers...
Running FCFS strategy...
Running Nearest-Driver strategy...
Running Nearest-Driver strategy...
Running Capacity-Aware strategy...
Running Capacity-Aware strategy...

✅ Simulation Complete!
Strategies evaluated: 3

✅ Simulation Complete!
Strategies evaluated: 3


In [16]:
# Display simulation results
# simulation_results is a DataFrame with strategy comparison
print("Strategy Comparison Results:")
print(simulation_results.to_string(index=False))

# Rename for plotting
metrics_df = simulation_results.copy()

Strategy Comparison Results:
      strategy  total_trips  on_time_rate  total_miles  avg_trip_duration  avg_idle_time  utilization_rate
          FCFS         4853       88.5400   23615.9300            21.1700             15           64.5100
       Nearest         4853       88.5400   23615.9300            21.1700             15           64.5100
Capacity-Aware         4853       88.5400   23615.9300            21.1700             15           64.5100


<a id="comparison"></a>
## 4. Strategy Comparison

Visual comparison of strategy performance.

In [17]:
# Key metrics comparison
fig = make_subplots(rows=1, cols=2, subplot_titles=('On-Time Rate (%)', 'Utilization Rate (%)'))

# On-Time Rate
fig.add_trace(
    go.Bar(x=metrics_df['strategy'], y=metrics_df['on_time_rate'], 
           marker_color=['#2E86AB', '#A23B72', '#F18F01'], name='On-Time %'),
    row=1, col=1
)

# Utilization Rate
fig.add_trace(
    go.Bar(x=metrics_df['strategy'], y=metrics_df['utilization_rate'],
           marker_color=['#2E86AB', '#A23B72', '#F18F01'], name='Utilization %'),
    row=1, col=2
)

fig.update_layout(title='Strategy Comparison: Key Metrics', showlegend=False, height=400)
fig.show()

In [18]:
# Radar chart comparing strategies
categories = ['On-Time Rate', 'Utilization', 'Avg Duration', 'Total Miles']

fig = go.Figure()

for _, row in metrics_df.iterrows():
    # Normalize values for radar chart (0-1 scale)
    values = [
        row['on_time_rate'] / 100,  # Already 0-100, convert to 0-1
        row['utilization_rate'] / 100,
        1 - (row['avg_trip_duration'] / 60),  # Lower is better, so invert
        1 - (row['total_miles'] / metrics_df['total_miles'].max())  # Lower is better
    ]
    values.append(values[0])  # Close the radar
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories + [categories[0]],
        name=row['strategy']
    ))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=True,
    title='Strategy Performance Radar'
)
fig.show()

In [19]:
# Bar chart comparing all metrics
metrics_to_plot = ['on_time_rate', 'utilization_rate', 'avg_trip_duration', 'avg_idle_time']
labels = ['On-Time Rate (%)', 'Utilization (%)', 'Avg Duration (min)', 'Avg Idle (min)']

fig = make_subplots(rows=2, cols=2, subplot_titles=labels)

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for metric, label, pos in zip(metrics_to_plot, labels, positions):
    fig.add_trace(
        go.Bar(
            x=metrics_df['strategy'], 
            y=metrics_df[metric],
            marker_color=['#2E86AB', '#A23B72', '#F18F01'],
            showlegend=False
        ),
        row=pos[0], col=pos[1]
    )

fig.update_layout(title='Strategy Metrics Comparison', height=600)
fig.show()

<a id="sensitivity"></a>
## 5. Sensitivity Analysis

Testing strategy performance under different conditions.

In [20]:
# Sensitivity: Vary number of available drivers
driver_counts = [50, 75, 100, 125, 150]
sensitivity_results = []

for n_drivers in driver_counts:
    print(f"Testing with {n_drivers} drivers...")
    results = run_simulation_comparison(active_trips.head(1000), num_drivers=n_drivers)
    
    for _, row in results.iterrows():
        sensitivity_results.append({
            'n_drivers': n_drivers,
            'strategy': row['strategy'],
            'on_time_rate': row['on_time_rate'],
            'utilization_rate': row['utilization_rate']
        })

sensitivity_df = pd.DataFrame(sensitivity_results)
print("\n✅ Sensitivity analysis complete!")

Testing with 50 drivers...
Running FCFS strategy...
Running Nearest-Driver strategy...
Running Capacity-Aware strategy...
Testing with 75 drivers...
Running FCFS strategy...
Running Capacity-Aware strategy...
Testing with 75 drivers...
Running FCFS strategy...
Running Nearest-Driver strategy...
Running Nearest-Driver strategy...
Running Capacity-Aware strategy...
Testing with 100 drivers...
Running FCFS strategy...
Running Capacity-Aware strategy...
Testing with 100 drivers...
Running FCFS strategy...
Running Nearest-Driver strategy...
Running Nearest-Driver strategy...
Running Capacity-Aware strategy...
Testing with 125 drivers...
Running FCFS strategy...
Running Capacity-Aware strategy...
Testing with 125 drivers...
Running FCFS strategy...
Running Nearest-Driver strategy...
Running Nearest-Driver strategy...
Running Capacity-Aware strategy...
Running Capacity-Aware strategy...
Testing with 150 drivers...
Running FCFS strategy...
Testing with 150 drivers...
Running FCFS strategy...
R

In [21]:
# Visualize sensitivity results
fig = px.line(
    sensitivity_df,
    x='n_drivers',
    y='on_time_rate',
    color='strategy',
    markers=True,
    title='On-Time Rate vs. Driver Pool Size',
    labels={'n_drivers': 'Number of Drivers', 'on_time_rate': 'On-Time Rate (%)'}
)
fig.show()

<a id="findings"></a>
## 6. Key Findings

### Strategy Comparison Summary

| Strategy | Pros | Cons |
|----------|------|------|
| **FCFS** | Simple, fast | Poor load balancing |
| **Nearest** | Minimizes travel | May overload nearby drivers |
| **Capacity-Aware** | Best balance | Higher computational cost |

### Recommendations

1. **For high-volume periods**: Use Capacity-Aware to prevent driver burnout
2. **For low-volume periods**: FCFS is sufficient and computationally cheap
3. **For geographic clustering**: Nearest works well in dense urban areas

### Limitations

> ⚠️ **Important**: This simulation compares assignment heuristics on historical data.
> It is NOT a real-time VRP optimizer. Production routing should use specialized 
> optimization engines (e.g., Google OR-Tools, OptaPlanner).

### Next Steps

- **Notebook 04**: Evaluation metrics and business impact analysis
- **Notebook 05**: Dashboard integration and deployment prep

In [22]:
# Export simulation results
output_path = PROCESSED_DIR / 'simulation_results.csv'
simulation_results.to_csv(output_path, index=False)
print(f"✅ Saved simulation results to {output_path}")

# Export sensitivity analysis
sensitivity_path = PROCESSED_DIR / 'sensitivity_analysis.csv'
sensitivity_df.to_csv(sensitivity_path, index=False)
print(f"✅ Saved sensitivity analysis to {sensitivity_path}")

print("\n✅ Data ready for notebook 04!")

✅ Saved simulation results to /Users/hc/Documents/projects/modivcare-rides-efficiency/data/processed/simulation_results.csv
✅ Saved sensitivity analysis to /Users/hc/Documents/projects/modivcare-rides-efficiency/data/processed/sensitivity_analysis.csv

✅ Data ready for notebook 04!
